In [ ]:
import os
import sys
print("Python version" + sys.version)
os.getcwd()
print(sys.executable)

In [ ]:
import numpy as np
np.random.seed(123)
import pandas as pd
import scipy
import itertools

import umap
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scanpy as sc
import anndata as ad
import scvelo as scv
from tqdm.notebook import tqdm

from pathlib import Path

In [3]:
from IPython.display import clear_output

In [4]:
from muon import prot as pt
from joblib import dump, load

In [5]:
import scrublet as scr

In [ ]:
sc.settings.verbosity = 1
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [7]:
# remove weird grid from scvelo
plt.rcParams['axes.grid'] = False

In [8]:
import itertools

In [9]:
from vcf_functions import *

In [10]:
from scitcem_utils import *

In [11]:
signatures_path_ = '../scrnaseq_signature_collection/'
from score_and_classify import *

### data input and variables

In [12]:
data_folder = '../data'

In [13]:
new_data_folder = '../processed_data'

In [14]:
tsamples = ['p007t', 'p008t', 'p009t1','p009t2', 'p013t', 'p014t', 'p016t', 
           'p020t', 'p021t', 'p026t', 'p035t'] 

In [15]:
samples = ['p007n', 'p008n', 'p009n1', 'p009n2','p013n', 'p014n', 'p016n', 
           'p020n', 'p021n'] + tsamples

In [16]:
MSI_list = ['p026', 'p035']

In [17]:
demux_sample = ['p020n', 'p021n', 'p020t', 'p021t']

### doublet detection by Scrublet and add h5 to an adata list

In [18]:
# for adata object that does not require demultiplexing by protein oligo tags
def get_adata(datafolder, sample, scrublet_threshold=None):

    # read
    adata = sc.read_10x_h5(datafolder/f'cellbender/{sample}/cellbender_counts.h5', gex_only=False)
    
    gene = adata.copy()
        
    gene.layers["CB_counts"] = gene.X.copy()
    
    gene.obs['sample'] = sample
    gene.obs['sample_origin'] = ['tumour' if sample[4:5] == 't' else 'normal'][0]
    gene.obs['patient'] = sample[:4]
    gene.obs['MS_status'] = ['MSI' if sample[:4] in MSI_list else 'MSS'][0]

    # sample + cell id
    gene.obs_names = [sample + ':' + x.split('-')[0] for x in gene.obs_names] # do this after getting cell id list
    gene.var_names_make_unique()
    
    # just empty row for concat
    gene.obs['target_hashtag'] = None
    gene.obs['second_hashtag'] = None
    gene.obs['high_prob_warm'] = None
    
    # scrublet score
    scrub = scr.Scrublet(gene.X)
        
    doublet_scores, predicted_doublets = scrub.scrub_doublets(get_doublet_neighbor_parents=False)

    if scrublet_threshold:
        predicted_doublets = scrub.call_doublets(threshold=scrublet_threshold)
        
    gene.obs[['doublet_score', 'predicted_doublet']] = pd.DataFrame({'doublet_score': doublet_scores, 
                                                                     'predicted_doublet': predicted_doublets}, 
                                                                    index=gene.obs_names)
    
    # add some QC plots
    scrub.plot_histogram()
    
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
    
    scrub.plot_embedding('UMAP', order_points=True)

    return gene

In [ ]:
# for adata object that requires demultiplexing by protein oligo tags
def get_adata_and_protein(datafolder, sample, target_hashtag, second_hashtag, scrublet_threshold=None):

    # read
    adata = sc.read_10x_h5(datafolder/f'cellbender/{sample}/cellbender_counts.h5', gex_only=False)
    
    gene = adata.copy()
        
    gene.layers["CB_counts"] = gene.X.copy()
    
    gene.obs['sample'] = sample
    gene.obs['sample_origin'] = ['tumour' if sample[4:5] == 't' else 'normal'][0]
    gene.obs['patient'] = sample[:4]
    gene.obs['MS_status'] = ['MSI' if sample[:4] in MSI_list else 'MSS'][0]

    
    # protein part
    protein = adata[:, adata.var["feature_types"] == "Antibody Capture"].copy()
    
    # normalise protein
    pt.pp.clr(protein)
    
    # add the two hashtags to adata.obs
    gene.obs['target_hashtag'] = pd.DataFrame(protein.X[:,target_hashtag].toarray(), columns = ['target_hashtag'],
                                           index=protein.obs.index).reindex(gene.obs.index)

    gene.obs['second_hashtag'] = pd.DataFrame(protein.X[:,second_hashtag].toarray(), columns = ['second_hashtag'],
                                           index=protein.obs.index).reindex(gene.obs.index)
    
    gene.obs['high_prob_warm'] = (gene.obs['target_hashtag'] >1) & (gene.obs['second_hashtag'] < 0.1)
    
    # sample + cell id
    gene.obs_names = [sample + ':' + x.split('-')[0] for x in gene.obs_names] # do this after getting cell id list
    gene.var_names_make_unique()
    
    # scrublet score
    scrub = scr.Scrublet(gene.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets(get_doublet_neighbor_parents=False)
    
    if scrublet_threshold:
        predicted_doublets = scrub.call_doublets(threshold=scrublet_threshold)
    
    gene.obs[['doublet_score', 'predicted_doublet']] = pd.DataFrame({'doublet_score': doublet_scores, 
                                                                     'predicted_doublet': predicted_doublets}, 
                                                                    index=gene.obs_names)

    # add some QC plots
    scrub.plot_histogram()
    
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
    
    scrub.plot_embedding('UMAP', order_points=True)
    
    return gene

In [20]:
adatas = {}

In [21]:
hashtag_list = [[1,3], [0,2], [10,11], [8,9]] 
# from patient metadata

In [22]:
# scrublet threshold after inspecting the hist
demux_threshold_list = 0.3

In [ ]:
for n in np.arange(0,4):
    gene = get_adata_and_protein(Path(data_folder), demux_sample[n],
                                 hashtag_list[n][0], hashtag_list[n][1], demux_threshold_list)
    adatas[demux_sample[n]] = gene

### add all the rest h5 files to the adata list and concat

In [24]:
rest_sample = samples.copy()
[rest_sample.remove(i) for i in rest_sample if i in demux_sample]
[rest_sample.remove(i) for i in rest_sample if i in demux_sample]

[None, None]

In [25]:
len(rest_sample)

16

In [26]:
rest_sample_threshold = 0.3

In [ ]:
for sample in rest_sample:
    gene = get_adata(Path(data_folder), sample, rest_sample_threshold)
    adatas[sample] = gene

In [29]:
adata_all = sc.concat(adatas)

### Mitochondria genes and pre-filtering QC plots

In [32]:
adata_all.var['mt'] = adata_all.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata_all, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata_all, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

### remove doublet, filter cells by min_counts and min_counts, filter mito genes, and post-filtering QC plots

In [36]:
adata_all = adata_all[adata_all.obs['predicted_doublet'] == False].copy()

In [38]:
sc.pp.filter_cells(adata_all, min_counts=1000)  
sc.pp.filter_cells(adata_all, min_genes=500) 

In [41]:
adata_all = adata_all[adata_all.obs.pct_counts_mt < 80, :]

In [42]:
# sanity
sc.pp.filter_genes(adata_all, min_cells=1)

In [ ]:
sc.pl.violin(adata_all, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

### normalisation, log, calculate cell cycle, ribo, hemo, and HVG

In [45]:
sc.pp.normalize_per_cell(adata_all)
sc.pp.log1p(adata_all)

In [46]:
score_cell_cycle(adata_all, signatures_path_)

In [ ]:
get_ribo_percentage(adata_all)
get_hemo_percentage(adata_all)

In [ ]:
sc.pp.highly_variable_genes(adata_all, n_top_genes=2000, batch_key='sample')

In [ ]:
sc.pl.highly_variable_genes(adata_all)

### calculate PCA, UMAP, louvain

In [ ]:
sc.tl.pca(adata_all, svd_solver='arpack', n_comps = 50, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata_all, log=False)

In [50]:
sc.pp.neighbors(adata_all, n_neighbors=50, n_pcs=20)
sc.tl.umap(adata_all)
sc.tl.diffmap(adata_all)
sc.tl.louvain(adata_all, key_added='louvain', resolution=1)
sc.tl.louvain(adata_all, key_added='louvain_highres', resolution=2)

### cell type annotation (coarse)

In [53]:
score_smillie_str_epi_imm(adata_all, signatures_path_)

In [54]:
adata_all.obs['cell_type_coarse'] = np.array(['epi', 'str', 'imm'])[np.argmax(adata_all.obs[['epi_score', 'str_score', 'imm_score']].values, axis=1)]
adata_all.obs['cell_type_coarse_score'] = np.max(adata_all.obs[['epi_score', 'str_score', 'imm_score']].values, axis=1)

In [ ]:
scv.pl.scatter(adata_all, basis='umap', color=['cell_type_coarse', 'epi_score', 'str_score', 'imm_score'], 
               ncols=2, dpi=300, legend_loc='right margin', size = 1)

### write h5

In [ ]:
adata_all.write(Path(new_data_folder)/'CB_all_cells.h5')

### write cell barcode id as anno/sample_celltype.txt for Numbat

In [59]:
celltype = ['epi', 'str', 'imm']

In [60]:
adata_copy = adata_all.copy()

In [61]:
adata_copy.obs_names = [x.split(':')[1] + '-1' for x in adata_copy.obs_names]

In [ ]:
for sample in samples:
    for ctype in celltype:
        (adata_copy[(adata_copy.obs['sample'] == sample) & (adata_copy.obs['celltype_1a'] == ctype)].
         obs_names.to_frame(name = 'cell_id').to_csv(Path(new_data_folder)/f'anno/{sample}_{ctype}.txt', 
                                                     index = False,
                                                     header = False))

In [62]:
del adata_copy

### focus on epithelial cells

In [208]:
adata_epi = adata_all[adata_all.obs['celltype_1a'] == 'epi'].copy()

### Calculate HVG, PCA, UMAP, louvain

In [ ]:
sc.pp.highly_variable_genes(adata_epi, n_top_genes=2000, batch_key='sample') 

In [ ]:
# default take HVG
sc.tl.pca(adata_epi, svd_solver='arpack', n_comps = 50, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata_epi, log=False)

In [214]:
sc.pp.neighbors(adata_epi, n_neighbors=20, n_pcs=15)
sc.tl.umap(adata_epi)
sc.tl.diffmap(adata_epi)
sc.tl.louvain(adata_epi, key_added='louvain', resolution=1)

In [ ]:
scv.pl.scatter(adata_epi, basis='umap', color=['sample_origin', 'sample', 
                                               'patient', 'MS_status', 
                                               'louvain'], 
               ncols=2, dpi=300, legend_loc='right margin', size = 2)

### write to h5 file

In [ ]:
adata_epi.write(Path(new_data_folder)/'CB_epi_cells.h5')